# Adrian Kamiński
## Praca domowa 3

### Wczytanie danych

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('australia.csv')

In [3]:
df

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,17.9,35.2,0.0,12.0,12.3,48.0,6.0,20.0,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,0,0
1,18.4,28.9,0.0,14.8,13.0,37.0,19.0,19.0,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,0,0
2,19.4,37.6,0.0,10.8,10.6,46.0,30.0,15.0,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,0,0
3,21.9,38.4,0.0,11.4,12.2,31.0,6.0,6.0,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,0,0
4,24.2,41.0,0.0,11.2,8.4,35.0,17.0,13.0,19.0,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56415,19.3,33.4,0.0,6.0,11.0,35.0,9.0,20.0,63.0,32.0,1013.9,1010.5,0.0,1.0,24.5,32.3,0,0
56416,21.2,32.6,0.0,7.6,8.6,37.0,13.0,11.0,56.0,28.0,1014.6,1011.2,7.0,0.0,24.8,32.0,0,0
56417,20.7,32.8,0.0,5.6,11.0,33.0,17.0,11.0,46.0,23.0,1015.3,1011.8,0.0,0.0,24.8,32.1,0,0
56418,19.5,31.8,0.0,6.2,10.6,26.0,9.0,17.0,62.0,58.0,1014.9,1010.7,1.0,1.0,24.8,29.2,0,0


### Podział na zbiór  trenigowy i testowy

In [4]:
X = df.drop('RainTomorrow', axis = 1)
y = df.RainTomorrow

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

### Uczenie 3 dowolnych klasyfikatorów

### Random Forest

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [8]:
rfc = RandomForestClassifier(
    n_estimators = 300, # liczba drzew w lesie 
    max_depth = 4       # maksymalna głębokość drzewa
)

In [9]:
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)

### Ada Boost Classifier

In [10]:
from sklearn.ensemble import AdaBoostClassifier

In [11]:
ada = AdaBoostClassifier(
    n_estimators = 70,  # maksymalna liczba estymatorów
    learning_rate = 1.1 # współczynnik uczenia
)

In [12]:
ada.fit(X_train, y_train)
ada_pred = ada.predict(X_test)

### Gradient Boosting Classifier

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

In [14]:
gbc = GradientBoostingClassifier(
    max_depth = 4,        # maksymalna głębokość pojedynczego estymatora
    min_samples_split = 3 # minimalna liczba próbek wymagana do podziału węzła
)

In [15]:
gbc.fit(X_train, y_train)
gbc_pred = gbc.predict(X_test)

### Ocena jakości klasyfikatorów

Wybór metryk:

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

In [17]:
metrics = [recall_score, roc_auc_score, accuracy_score, precision_score]

In [18]:
models = [(rfc_pred, "RandomForestClassifier"),(ada_pred, "AdaBoostClassifier"), (gbc_pred, 'GradientBoostingClassifier')]

In [19]:
data = {}
for pred, name in models:
    output = []
    for i in range(len(metrics)):
        output.append(metrics[i](y_test, pred))
    data[name] = output

results = pd.DataFrame(data, index = [metric.__name__ for metric in metrics]).transpose()

In [20]:
results.sort_values(
    # uznałem, że najważniejsze jest zminializowanie false negative'ów stąd recall na pierwszym miejscu
    by = ['recall_score', 'roc_auc_score', 'accuracy_score', 'precision_score'], 
    ascending = False)

,recall_score,roc_auc_score,accuracy_score,precision_score
GradientBoostingClassifier,0.532797,0.740657,0.856966,0.745076
AdaBoostClassifier,0.517907,0.731280,0.850674,0.725479
RandomForestClassifier,0.400000,0.686192,0.846331,0.803557


Najlepsze wyniki osiąga <ins>GradientBoostingClassifier</ins> (najwyższy wynik przy większości z wybranych metryk).